For røykerandelen av befolkningen
https://www.ssb.no/helse/helseforhold-og-levevaner/statistikk/royk-alkohol-og-andre-rusmidler

For luftkvalitet
https://luftkvalitet.nilu.no/historikk



Potensielt 
https://norgeshelsa.no/norgeshelsa/

## Datakilde
- statbank
- asthma og status asthmaticus (astma og akutt astma)
- per nå for alle år, alle kjønn, alle aldere (gruppevis) for både liggedager, dagbehandlinger og utksrivninger.

https://www.ssb.no/en/statbank/table/03672/tableViewLayout1/

In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("processed_data.xlsx")
df.head(15)

,Age group,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008
0,"Years, total",7.0,6.4,6.6,4.9,4.2,3.8,3.5,4.9,4.9,5.2
1,0-9 years,15.8,11.6,12.5,8.5,7.7,7.2,6.9,9.6,8.4,7.8
2,10-19 years,10.5,11.4,12.1,2.0,1.6,1.5,1.3,6.6,7.1,8.9
3,20-39 years,3.0,2.5,2.4,2.5,2.0,2.0,1.7,1.7,1.7,1.8
4,40-59 years,4.6,4.8,5.9,5.3,4.6,4.8,4.2,4.3,4.5,5.1
5,60-69 years,6.5,6.1,7.1,7.5,5.9,4.2,4.6,5.9,6.4,6.1
6,70-79 years,8.9,8.4,6.5,8.3,7.0,5.0,4.6,6.7,6.2,6.8
7,80 years or older,6.6,7.4,4.7,6.1,5.3,3.6,3.8,4.7,4.6,4.0
8,Unknown age,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"Years, total",6.7,5.5,5.3,5.3,4.4,4.2,3.5,4.5,6.6,5.5


In [2]:

all_rows = ["Total", "0-9 years", "10-19 years", "20-39 years",
            "40-59 years", "60-69 years", "70-79 years",
            "80 years or older", "Unknown age"]


In [3]:
age_col = None
if isinstance(df.columns, pd.MultiIndex):
    for col in df.columns:
        # try to detect a column whose values match your age labels
        if df[col].astype(str).str.strip().isin(all_rows).any():
            age_col = col
            break
else:
    # common single-level names
    for cand in ("Age group", "age group", "age", "group"):
        if cand in df.columns:
            age_col = cand
            break
    # fallback: any string column that overlaps labels
    if age_col is None:
        for c in df.columns:
            if df[c].dtype == object and df[c].astype(str).str.strip().isin(all_rows).any():
                age_col = c
                break

In [4]:
if age_col is None:
    raise KeyError("Could not find a column containing the age-group labels")

# normalized label column used for grouping
df["_age_label"] = df[age_col].astype(str).str.strip()

# sum numeric columns per age label
numeric_cols = df.select_dtypes(include=[np.number]).columns
grouped = df.groupby("_age_label")[numeric_cols].sum()

# ensure the desired order and fill missing with 0
grouped = grouped.reindex(all_rows[1:], fill_value=0)

# create Total row (sum of all groups) and prepend
total = grouped.sum(axis=0).to_frame().T
total.index = ["Total"]
result = pd.concat([total, grouped])

In [5]:
display(result)

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008
Total,1073.7,1006.9,987.3,835.8,673.8,572.3,548.3,745.8,746.5,747.6
0-9 years,290.9,212.0,223.1,158.2,143.5,143.2,129.9,179.1,160.7,142.6
10-19 years,176.9,188.6,198.2,38.5,31.4,29.4,28.5,101.3,109.4,136.0
20-39 years,66.1,51.4,48.7,52.9,46.3,41.2,36.1,37.2,37.4,42.2
40-59 years,91.0,104.4,125.4,116.3,100.6,98.6,87.5,85.4,92.0,103.1
60-69 years,134.6,131.8,153.3,166.2,119.9,88.5,92.0,124.6,131.4,127.3
70-79 years,181.2,171.5,150.4,158.1,132.2,98.4,102.3,127.7,125.5,124.6
80 years or older,133.0,147.2,88.2,145.6,99.9,73.0,72.0,90.5,90.1,71.8
Unknown age,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
